In [9]:
import json
from openai import OpenAI
from gemini import GeminiService
import logging
from datetime import datetime
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=log_format)
log = logging.getLogger()
log.setLevel(logging.DEBUG)

clean = True
openai = False

In [10]:

SYSTEM_PROMPT = """
**You are a UPSC prelims question expert specializing in converting English MCQs to Hindi.**

**Given a question in English (statement) and a hint, convert it to Hindi (statement) suitable for the UPSC prelims exam, maintaining technical accuracy and UPSC context.**

**Input (JSON):**

* `statement`: Question statement with options in English.
* `hint`: Hint for the question in English.

**Output (JSON):**

* `statement`: Converted question statement with choices at end in Hindi.
* `hint`: Hint for the question in Hindi.

**IMPORTANT INFORMATION**
- DONOT change the base statement format. The answers number should remain the same in the converted statement.
- DONOT forget to translate choices from end of each input `statement` to reponse `statement`.
- Maintain line breaks `\\n` in response similar to input
"""


In [11]:
import re

def clean_text(text, clean=True):
    if not clean:
        return re.sub(r'\s+', ' ',text.strip())
        # return text.strip()
    return re.sub(r'\s+', ' ',text.replace("\n"," ").strip())

In [12]:

input_data = []
with open ('data.json', 'r') as fp:
    json_data = json.load(fp)
for index, (key, item) in enumerate(json_data.items()):
    if index > 3:
        break
    statement = clean_text(item['statement'], clean)
    hint = clean_text(item['hint'], clean)
    temp = {'statement': statement, 'hint': hint}
    input_data.append(temp)

In [13]:
input_data

[{'statement': 'Consider following statements regarding the representation of States in the Parliament: 1. Delimitation of Constituencies is undertaken on the basis of census exercise to ensure that every State is represented in proportion to its population in both the Houses of Parliament. 2. Delimitation Commission is a constitutional body, the notification of whose orders cannot be challenged in a Court. 3. Territorial constituencies in States, at present, are based on the data of 2001 census, as the Constitution (87thAmendment) Act, 2003 enabled the delimitation exercise on the basis of 2001Census figures. 4. As it stands today, Constitution of India prohibits any delimitation exercise till 2031. Which of the statements given above are not correct ? (A) 1, 2 and 4only (B) 2, 3 and 4only (C) 1, 3 and 4only (D) 1, 2, 3 and4',
  'hint': 'Delimitation constituencies are NOT applicable to representation of states in Council of States. Though it is correct to say the Order of delimitatio

In [14]:
reponse_data = []
if openai:
    for i in input_data:
        USER_PROMPT = str(i)
        data = OpenAI.generate_completion(SYSTEM_PROMPT, USER_PROMPT)
        reponse_data.append(data)
else:
    gem = GeminiService(SYSTEM_PROMPT)
    for i in input_data:
        USER_PROMPT = str(i)
        data = gem.generate_completion(USER_PROMPT)
        reponse_data.append(data)
        break

    

2024-04-22 15:28:39,728 - INFO - Sending request to gemini api.
2024-04-22 15:28:55,005 - INFO - Request successfull.


<class 'dict'>


In [15]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
service = "openai" if openai else 'gemini'

with open(f'response/{service}-response-{now}.json', 'w') as fp:
    json.dump(reponse_data, fp, ensure_ascii=False)


In [16]:
print(input_data[1])
print(reponse_data[1])

{'statement': 'Which of the following statements is/are true about the Gram Sabha? 1. All people living in a village or a group of villages are members of the Gram Sabha. 2. All the plans for work of Gram Panchayat have to be approved by Gram Sabha. 3. For better implementation of some specific tasks, Gram Sabha form committees. 4. The elected Secretary of the Gram Sabha calls the meeting and keeps a record of the proceedings. (A) 2 and 3 (B) 1, 3 and 4 (C) 2, 3 and 4 (D) 1,2,3,4', 'hint': "Only adult villagers who have the right to vote can be member of Gram Sabha. Persons below 18 years of age can't become members. Gram Sabha plays a supervisory and monitoring role over Gram Panchayat by approving it plan of work. Gram Sabha form committees like construction, animal husbandry, etc to carry out some specific tasks. The Gram Panchayat has a Secretary who is also the Secretary of the Gram Sabha. This person is not an elected person but is appointed by the government. The Secretary is re

IndexError: list index out of range

In [ ]:
example_data = []
filename = 'example_stripped' if clean else 'example_not_stripped'
service = "openai" if openai else 'gemini'
now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

for x,y in zip(input_data,reponse_data):
    example_data.append([x,y])

with open(f'example/{service}-{filename}-{now}.json', 'w') as fp:
    json.dump(example_data, fp, ensure_ascii=False)
